<a href="https://colab.research.google.com/github/meetreks/AWS/blob/master/SaiSpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz

In [ ]:
!tar xf spark-3.0.1-bin-hadoop3.2.tgz

In [ ]:
!pip install -q findspark

In [ ]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
sc = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import sys, tempfile, urllib.request
from pyspark.sql.functions import *

In [ ]:
BASE_DIR = '/tmp'
CORONA_DATA_FILE = os.path.join(BASE_DIR,'corona_data.csv')
corona_data = urllib.request.urlretrieve("https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/corona_dataset_latest.csv", CORONA_DATA_FILE)

In [ ]:
BASE_DIR = '/tmp'
TWITTER_DATA_FILE = os.path.join(BASE_DIR,'twitter_data.csv')
tweet_data = urllib.request.urlretrieve("https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/tweets.csv", TWITTER_DATA_FILE)

In [ ]:
!ls /tmp

blockmgr-3f702a38-07fd-48b8-b5b9-0c9340064cb3
corona_data.csv
hsperfdata_root
liblz4-java-8368312585355798075.so
liblz4-java-8368312585355798075.so.lck
spark-8738110c-e27e-4ea1-af60-7438835ee207
spark-bb6f22d8-fce3-4d40-93d5-3d8257a2e872
twitter_data.csv


In [ ]:
corna_df = sc.read.option("inferschema", "true").csv("/tmp/corona_data.csv", header = True)

In [ ]:
corna_df.show()

+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|    112.5|2020-01-22|        0|    0|        0|             Sarawa

In [ ]:
corna_df.count()

28143

In [ ]:
twitter_df = sc.read.option("inferschema", "true").csv("/tmp/twitter_data.csv", header = True)

In [ ]:
twitter_df.show()

+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0| geo|                text|                user|            location|            entities|           sentiment|             country|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|null|What is God sayin...|          petodinice|               Lagos|[('about #', 'CAR...|{'neg': 0.0, 'neu...|             Nigeria|
|  1|null|"BREAKING: ""this...| but i took the t...|                -… "|     JerryfranksonJF|      Abuja, Nigeria|"[(""Arsenal's Mi...|
|  2|null| #Coronavirus tes...|              cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|                 USA|
|  3|null| Get ready for ma...|        InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|                 USA|
|  4|null| The #coronavirus...|          

In [ ]:
twitter_df.count()

1000

In [ ]:
twitter_df.filter("country='USA'").show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|  2|null| #Coronavirus tes...|         cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|    USA|
|  3|null| Get ready for ma...|   InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|    USA|
|  5|null| COVID-19 update ...| StewartNgilana|Durban | Port Eli...|[('Italy', 'GPE')...|{'neg': 0.178, 'n...|    USA|
|  6|null| It’s painful to ...|      BWheatnyc|             Florida|                  []|{'neg': 0.098, 'n...|    USA|
|  8|null| Questions about ...|    straightj23|        Columbus, OH|[('NAfME', 'CARDI...|{'neg': 0.0, 'neu...|    USA|
| 13|null| We’re the heck w...| harrytiffanyiv| 

In [ ]:
twitter_df.filter("country='USA' and location like '%USA%'").show()

+---+----+--------------------+--------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|          user|            location|            entities|           sentiment|country|
+---+----+--------------------+--------------+--------------------+--------------------+--------------------+-------+
|  2|null| #Coronavirus tes...|        cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|    USA|
| 18|null| Wash your hands....|  Mrrandy123RP|          Texas, USA|            ['', '']|{'neg': 0.0, 'neu...|    USA|
| 31|null| I ordered Alex J...|      rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|    USA|
| 62|null|Fear will kill #C...|RobertPPurcell|norther New Jerse...|                ['']|{'neg': 0.798, 'n...|    USA|
|105|null| Does anybody els...|         bremi|   Pennsylvania, USA|[('approximately ...|{'neg': 0.254, 'n...|    USA|
|134|null| This #coronaviru...|   fringephile|       Okl

In [ ]:
twitter_df.explain()

== Physical Plan ==
FileScan csv [_c0#126,geo#127,text#128,user#129,location#130,entities#131,sentiment#132,country#133] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [ ]:
x = sc.sparkContext.parallelize([1,2,3,4,5,6,7,8,9])
y = x.map(lambda x: (x, x*x))
print(x.collect())
print(y.collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9]
[(1, 1), (2, 4), (3, 9), (4, 16), (5, 25), (6, 36), (7, 49), (8, 64), (9, 81)]


In [ ]:
x = sc.sparkContext.parallelize([1,2,3,4,5,6,7,8,9])
y = x.flatMap(lambda x: (x, x*x))
print(x.collect())
print(y.collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 1, 2, 4, 3, 9, 4, 16, 5, 25, 6, 36, 7, 49, 8, 64, 9, 81]


In [ ]:
corna_df.filter("Country = 'US'").sort(col("Date"), ascending=False).show()
corna_df.sort(col("Confirmed"), ascending=False).show()

+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20|     1177|   23|        0|      California|      California|
|27767|   Massachusetts|     US|           42.2302| -71.5301|2020-03-

In [ ]:
corna_df.agg({"Date": "max", "confirmed": "max"}).collect()

[Row(max(confirmed)=67800, max(Date)='2020-03-20')]

In [ ]:
!pip install apache-airflow

In [ ]:
airflow initdb

In [ ]:
!airflow initdb

In [ ]:
!airflow webserver -p 8080

In [ ]:
!airflow scheduler

In [ ]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

ws=Window().partitionBy("Country", "state_cleaned").orderBy(col("Date").desc())
corna_df.withColumn("row_num", row_number().over(ws)).show()

In [ ]:
corna_df.withColumn("row_num", row_number().over(ws)).where(col("row_num")==1).show()

+-----+--------------------+--------------+------------------+---------+----------+---------+-----+---------+--------------------+--------+-------+
|  _c0|               State|       Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|    City|row_num|
+-----+--------------------+--------------+------------------+---------+----------+---------+-----+---------+--------------------+--------+-------+
|27745|                null|      Cameroon|3.8480000000000003|  11.5021|2020-03-20|       20|    0|        0|             Yaounde|    null|      1|
|27859|             Qinghai|         China|           35.7452|  95.9956|2020-03-20|       18|    0|       18|             Qinghai|    null|      1|
|27762|                null|        Cyprus|           35.1264|  33.4299|2020-03-20|       67|    0|        0|             Nicosia|    null|      1|
|27812|            Michigan|            US|           43.3266| -84.5361|2020-03-20|      552|    3|        0|   

In [ ]:
!pip install apache-beam

     |████████████████████████████████| 8.7MB 2.6MB/s 
     |████████████████████████████████| 63.8MB 110kB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 1.4MB 40.0MB/s 
     |████████████████████████████████| 829kB 44.1MB/s 
     |████████████████████████████████| 153kB 41.9MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 112kB 49.5MB/s 
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-cp36-none-any.whl size=43516 sha256=0c924ff9789dc4ce8c4a53b38a5c346b1631aa945538e649e44822561e3ea50e
  Stored in directory: /root/.cache/pip/wheels/5a/d3/be/86620c9dd3fca68986c33b9c616510289fc0abb81ec9aa70bd
  Created wheel for hdfs: filename=hdfs-2.5.8-cp36-none-any.whl size=33213 sha256=593b151b9ff19976a2b1f436a37dbe5d76d3e36b7c9b6ff2b985a9a729249a19
  Stored in directory: /root/.cache/pip/wheels/fe/a7/05/23e3699975fc20f8a30e00ac1e51

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
import re
import os

In [3]:
inputs_pattern = '/content/sample_data/inpt.txt'
outputs_prefix = '/content/sample_data/outputs/part1'

In [4]:
options = PipelineOptions([
    "--runner=FlinkRunner",
    "--flink_version=1.8",
    "--flink_master=localhost:8081",
    "--environment_type=LOOPBACK"
])

In [7]:
with beam.Pipeline(options=options) as pipeline:
    (
            pipeline
            | 'Read lines' >> beam.io.ReadFromText(inputs_pattern)
            | 'Find words' >> beam.FlatMap(lambda line: re.findall(r"[a-zA-Z']+", line))
            | 'Pair words with 1' >> beam.Map(lambda word: (word, 1))
            | 'Group and sum' >> beam.CombinePerKey(sum)
            | 'Format results' >> beam.Map(lambda word_count: str(word_count))
            | 'Write results' >> beam.io.WriteToText(outputs_prefix)
    )